# Image classification 

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

### Dataset preprocessing 

In [2]:
dataset_path = "data_set"
img_size = (224, 224)
batch_size = 8

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=5,
    zoom_range=0.1,
    brightness_range=[0.9, 1.1]
)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 75 images belonging to 8 classes.
Found 16 images belonging to 8 classes.


### Model building

In [3]:
base_model = MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base_model.trainable = False  # Freeze base model

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


### Model training

In [4]:
model.fit(train_data, validation_data=val_data, epochs=10)


d:\int2\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 628ms/step - accuracy: 0.2223 - loss: 2.3391

d:\int2\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.2348 - loss: 2.2894 - val_accuracy: 0.5625 - val_loss: 1.2832
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 551ms/step - accuracy: 0.6994 - loss: 0.9971 - val_accuracy: 0.7500 - val_loss: 0.7058
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 558ms/step - accuracy: 0.9387 - loss: 0.3394 - val_accuracy: 0.8125 - val_loss: 0.5096
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 547ms/step - accuracy: 0.9876 - loss: 0.2641 - val_accuracy: 0.7500 - val_loss: 0.6428
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 560ms/step - accuracy: 1.0000 - loss: 0.1425 - val_accuracy: 0.7500 - val_loss: 0.4790
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 583ms/step - accuracy: 0.9914 - loss: 0.1055 - val_accuracy: 0.9375 - val_loss: 0.3835
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 610ms/step - accuracy: 1.0000 - loss: 0.0673 - val_accuracy: 0.8750 - val_loss: 0.4722
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 642ms/step - accuracy: 1.0000 - loss: 0.0443 - val_accuracy: 0.8125 - val_lo

In [5]:
model.save("youtube_page_classifier.h5")

### Prediction fuction

In [6]:
folder_to_logical = {
    "001_login": "login",
    "002_home": "home",
    "004_channel": "channel",
    "0031_search": "search",
    "0032_history": "history",
    "0032_music": "music",
    "0033_shorts": "shorts",
    "0034_Video_Page": "video_page"
}




def predict_image(image_path):
    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    confidence = np.max(prediction)

    class_labels = list(train_data.class_indices.keys())  # like ['0032_music', ...]
    predicted_folder = class_labels[class_idx]
    predicted_label = folder_to_logical.get(predicted_folder, "unknown")

    print(f"Predicted Page Type: {predicted_label} (Confidence: {confidence:.2f})")
    return predicted_label 


### Result

In [ ]:
start_page_detected = predict_image("test1.png")  




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted Page Type: login (Confidence: 0.91)



## Navigation Flow 



In [23]:

import collections
destination_page = "history"


youtube_map = {
    'login': ['home'],
    'home': ['search', 'history', 'music', 'shorts', 'video_page', 'channel'],
    'channel': ['home', 'video_page'],
    'search': ['video_page', 'channel', 'shorts', 'home'],
    'history': ['video_page', 'home'],
    'music': ['home', 'video_page'],
    'shorts': ['home', 'video_page', 'channel'],
    'video_page': ['home', 'channel', 'shorts']
}

# BFS
def find_shortest_path(graph, start, end):
    if start not in graph or end not in graph:
        return None
    queue = collections.deque([(start, [start])])
    visited = {start}
    while queue:
        current_node, path = queue.popleft()
        for neighbor in graph.get(current_node, []):
            if neighbor not in visited:
                visited.add(neighbor)
                new_path = list(path)
                new_path.append(neighbor)
                if neighbor == end:
                    return new_path
                queue.append((neighbor, new_path))
    return None


path = find_shortest_path(youtube_map, start_page_detected, destination_page)

if path:
    print(f"\nNavigation Path from '{start_page_detected}' to '{destination_page}':")
    print(" -> ".join(path))
else:
    print(f"\n No valid path found from '{start_page_detected}' to '{destination_page}'.")





Navigation Path from 'login' to 'history':
login -> home -> history
